In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Advanced Pinecone Usage\n",
    "\n",
    "This notebook demonstrates advanced techniques with Pinecone vector database, including:\n",
    "- Working with multiple indexes\n",
    "- Multi-modal vectors\n",
    "- Advanced metadata filtering\n",
    "- Query-time vector transformations\n",
    "- Performance optimization\n",
    "- Index monitoring\n",
    "\n",
    "Let's start by setting up our environment."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import os\n",
    "import time\n",
    "import uuid\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from typing import List, Dict, Any, Optional\n",
    "from sklearn.decomposition import PCA\n",
    "from sentence_transformers import SentenceTransformer\n",
    "\n",
    "from pinecone import Pinecone, ServerlessSpec\n",
    "from src.config import (DEFAULT_CLOUD, DEFAULT_DIMENSION, DEFAULT_METRIC, DEFAULT_REGION)\n",
    "from src.utils import (get_pinecone_client, create_random_vectors, wait_for_index_ready, clean_up_index)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Working with Multiple Indexes\n",
    "\n",
    "In some applications, you may need to work with multiple indexes for different types of data or different embedding models. Here's how to manage multiple indexes efficiently."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def create_multiple_indexes(count: int = 2, dims: List[int] = None):\n",
    "    \"\"\"Create multiple indexes with different dimensions.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    \n",
    "    # Use default dimension if none provided\n",
    "    if not dims:\n",
    "        dims = [DEFAULT_DIMENSION] * count\n",
    "    \n",
    "    index_names = []\n",
    "    \n",
    "    for i in range(count):\n",
    "        # Create a unique index name\n",
    "        unique_id = str(uuid.uuid4())[:8]\n",
    "        index_name = f\"multi-index-{i}-{unique_id}\"\n",
    "        \n",
    "        # Check if index already exists\n",
    "        if pc.has_index(index_name):\n",
    "            print(f\"Index {index_name} already exists, skipping creation\")\n",
    "            index_names.append(index_name)\n",
    "            continue\n",
    "            \n",
    "        # Create a new serverless index with specified dimension\n",
    "        print(f\"Creating index {i+1}/{count}: {index_name} with dimension {dims[i]}\")\n",
    "        pc.create_index(\n",
    "            name=index_name,\n",
    "            vector_type=\"dense\",\n",
    "            dimension=dims[i],\n",
    "            metric=DEFAULT_METRIC,\n",
    "            spec=ServerlessSpec(\n",
    "                cloud=DEFAULT_CLOUD,\n",
    "                region=DEFAULT_REGION\n",
    "            )\n",
    "        )\n",
    "        \n",
    "        # Wait for the index to be ready\n",
    "        wait_for_index_ready(pc, index_name)\n",
    "        index_names.append(index_name)\n",
    "    \n",
    "    return index_names\n",
    "\n",
    "# Create two indexes with different dimensions\n",
    "index_names = create_multiple_indexes(2, [384, 768])  # Common dimensions for different models\n",
    "print(f\"Created indexes: {index_names}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Populating Each Index with Appropriate Vectors"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def populate_indexes(index_names: List[str], dimensions: List[int], vector_count: int = 50):\n",
    "    \"\"\"Populate each index with appropriate dimensional vectors.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    \n",
    "    for idx, (index_name, dim) in enumerate(zip(index_names, dimensions)):\n",
    "        index = pc.index(index_name)\n",
    "        \n",
    "        # Create random vectors with the appropriate dimension\n",
    "        vectors = create_random_vectors(vector_count, dim)\n",
    "        \n",
    "        # Prepare vector data with ids and metadata\n",
    "        vector_data = [\n",
    "            {\n",
    "                \"id\": f\"vec-idx{idx}-{i}\",\n",
    "                \"values\": vectors[i],\n",
    "                \"metadata\": {\n",
    "                    \"index\": idx,\n",
    "                    \"dimension\": dim,\n",
    "                    \"category\": f\"category-{i % 3}\",\n",
    "                    \"priority\": i % 5,\n",
    "                    \"timestamp\": time.time() - (i * 3600)  # Different timestamps\n",
    "                }\n",
    "            }\n",
    "            for i in range(vector_count)\n",
    "        ]\n",
    "        \n",
    "        # Upsert in batches\n",
    "        batch_size = 25\n",
    "        for i in range(0, len(vector_data), batch_size):\n",
    "            batch = vector_data[i:i+batch_size]\n",
    "            index.upsert(vectors=batch)\n",
    "            print(f\"Index {index_name}: Upserted vectors {i} to {i+len(batch)-1}\")\n",
    "            \n",
    "        # Check vector count\n",
    "        time.sleep(2)  # Allow time for indexing\n",
    "        stats = index.describe_index_stats()\n",
    "        print(f\"Index {index_name}: Total vectors: {stats.namespaces.get('', {}).get('vector_count', 0)}\")\n",
    "\n",
    "# Populate the indexes\n",
    "populate_indexes(index_names, [384, 768], 50)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Multi-Modal Vectors\n",
    "\n",
    "Multi-modal vectors involve using different embedding models for different types of content (text, images, etc.). Here we'll simulate working with multi-modal data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def simulate_multimodal_vectors(text_samples: List[str]):\n",
    "    \"\"\"Simulate text and image vectors using SentenceTransformer.\"\"\"\n",
    "    # Create a sentence transformer model for text embeddings\n",
    "    text_model = SentenceTransformer('all-MiniLM-L6-v2')  # 384 dimensions\n",
    "    \n",
    "    # Get text embeddings\n",
    "    text_embeddings = text_model.encode(text_samples)\n",
    "    \n",
    "    # Simulate image embeddings with random vectors (in real apps, you'd use a vision model)\n",
    "    image_embeddings = create_random_vectors(len(text_samples), 512)\n",
    "    \n",
    "    return {\n",
    "        \"text\": text_embeddings.tolist(),\n",
    "        \"image\": image_embeddings\n",
    "    }\n",
    "\n",
    "# Sample text data\n",
    "text_samples = [\n",
    "    \"A red sports car parked by the beach\",\n",
    "    \"A dog playing in the park\",\n",
    "    \"A cityscape at night with bright lights\",\n",
    "    \"A plate of pasta with tomato sauce\",\n",
    "    \"A person hiking on a mountain trail\"\n",
    "]\n",
    "\n",
    "# Generate embeddings\n",
    "multimodal_vectors = simulate_multimodal_vectors(text_samples)\n",
    "\n",
    "# Print shapes\n",
    "print(f\"Text vectors shape: {len(multimodal_vectors['text'])} x {len(multimodal_vectors['text'][0])}\")\n",
    "print(f\"Image vectors shape: {len(multimodal_vectors['image'])} x {len(multimodal_vectors['image'][0])}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Creating a Multi-Modal Index and Upserting Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def create_multimodal_index():\n",
    "    \"\"\"Create an index for multi-modal vectors.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    \n",
    "    # Create a unique index name\n",
    "    unique_id = str(uuid.uuid4())[:8]\n",
    "    index_name = f\"multimodal-{unique_id}\"\n",
    "    \n",
    "    # Check if index already exists\n",
    "    if pc.has_index(index_name):\n",
    "        print(f\"Index {index_name} already exists, skipping creation\")\n",
    "        return index_name\n",
    "    \n",
    "    # For multi-modal, we'll use a dimension that can fit all our vectors\n",
    "    # We'll use a technique called \"vector concatenation\" for demonstration\n",
    "    # In real applications, you might use separate indexes or more sophisticated techniques\n",
    "    \n",
    "    # Create a new serverless index with dimension that fits both text and image vectors\n",
    "    # We'll pad the smaller vectors\n",
    "    combined_dim = 1024  # Large enough for both types after padding\n",
    "    \n",
    "    print(f\"Creating multi-modal index: {index_name}\")\n",
    "    pc.create_index(\n",
    "        name=index_name,\n",
    "        vector_type=\"dense\", \n",
    "        dimension=combined_dim,\n",
    "        metric=DEFAULT_METRIC,\n",
    "        spec=ServerlessSpec(\n",
    "            cloud=DEFAULT_CLOUD,\n",
    "            region=DEFAULT_REGION\n",
    "        )\n",
    "    )\n",
    "    \n",
    "    # Wait for the index to be ready\n",
    "    wait_for_index_ready(pc, index_name)\n",
    "    return index_name\n",
    "\n",
    "def pad_vector(vector: List[float], target_dim: int) -> List[float]:\n",
    "    \"\"\"Pad a vector to the target dimension with zeros.\"\"\"\n",
    "    if len(vector) >= target_dim:\n",
    "        return vector[:target_dim]\n",
    "    return vector + [0.0] * (target_dim - len(vector))\n",
    "\n",
    "def upsert_multimodal_data(index_name: str, text_vectors: List[List[float]], \n",
    "                          image_vectors: List[List[float]], descriptions: List[str]):\n",
    "    \"\"\"Upsert multi-modal vectors with appropriate metadata.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    index = pc.index(index_name)\n",
    "    \n",
    "    target_dim = 1024\n",
    "    vector_data = []\n",
    "    \n",
    "    # Create text vector records\n",
    "    for i, (text_vec, desc) in enumerate(zip(text_vectors, descriptions)):\n",
    "        padded_vec = pad_vector(text_vec, target_dim)\n",
    "        vector_data.append({\n",
    "            \"id\": f\"text-{i}\",\n",
    "            \"values\": padded_vec,\n",
    "            \"metadata\": {\n",
    "                \"type\": \"text\",\n",
    "                \"description\": desc,\n",
    "                \"original_dim\": len(text_vec)\n",
    "            }\n",
    "        })\n",
    "    \n",
    "    # Create image vector records\n",
    "    for i, (img_vec, desc) in enumerate(zip(image_vectors, descriptions)):\n",
    "        padded_vec = pad_vector(img_vec, target_dim)\n",
    "        vector_data.append({\n",
    "            \"id\": f\"image-{i}\",\n",
    "            \"values\": padded_vec,\n",
    "            \"metadata\": {\n",
    "                \"type\": \"image\",\n",
    "                \"description\": desc,\n",
    "                \"original_dim\": len(img_vec)\n",
    "            }\n",
    "        })\n",
    "    \n",
    "    # Upsert all vector data\n",
    "    index.upsert(vectors=vector_data)\n",
    "    print(f\"Upserted {len(vector_data)} multi-modal vectors\")\n",
    "    \n",
    "    # Check vector count\n",
    "    time.sleep(2)  # Allow time for indexing\n",
    "    stats = index.describe_index_stats()\n",
    "    print(f\"Total vectors in index: {stats.namespaces.get('', {}).get('vector_count', 0)}\")\n",
    "    \n",
    "    return vector_data\n",
    "\n",
    "# Create multimodal index\n",
    "multimodal_index = create_multimodal_index()\n",
    "\n",
    "# Upsert multimodal data\n",
    "multimodal_data = upsert_multimodal_data(\n",
    "    multimodal_index, \n",
    "    multimodal_vectors['text'], \n",
    "    multimodal_vectors['image'],\n",
    "    text_samples\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Advanced Metadata Filtering\n",
    "\n",
    "Pinecone allows for sophisticated metadata filtering. Let's explore complex filter expressions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def advanced_metadata_filtering(index_name: str):\n",
    "    \"\"\"Demonstrate advanced metadata filtering techniques.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    index = pc.index(index_name)\n",
    "    \n",
    "    # Get a random vector to use as query\n",
    "    query_vector = create_random_vectors(1, 1024)[0]\n",
    "    \n",
    "    print(\"\\n--- Advanced Metadata Filtering Examples ---\")\n",
    "    \n",
    "    # 1. Filter by exact match\n",
    "    filter_exact = {\"type\": \"text\"}\n",
    "    results_exact = index.query(\n",
    "        vector=query_vector,\n",
    "        filter=filter_exact,\n",
    "        top_k=3,\n",
    "        include_metadata=True\n",
    "    )\n",
    "    print(\"\\n1. Filter by exact match (type='text'):\")\n",
    "    for i, match in enumerate(results_exact.matches):\n",
    "        print(f\"  {i+1}. ID: {match.id}, Score: {match.score:.4f}, Metadata: {match.metadata}\")\n",
    "    \n",
    "    # 2. Range filtering with $gt, $gte, $lt, $lte\n",
    "    # In this example we'll use the timestamps we'd create in a real scenario\n",
    "    # Create some timestamp-based vector data first\n",
    "    timestamp_vectors = []\n",
    "    current_time = time.time()\n",
    "    for i in range(20):\n",
    "        # Create vectors with timestamps spread over the last 30 days\n",
    "        timestamp = current_time - (i * 86400 * 1.5)  # Approx. 1.5 days apart\n",
    "        timestamp_vectors.append({\n",
    "            \"id\": f\"time-{i}\",\n",
    "            \"values\": create_random_vectors(1, 1024)[0],\n",
    "            \"metadata\": {\n",
    "                \"timestamp\": timestamp,\n",
    "                \"priority\": i % 5,\n",
    "                \"category\": f\"cat-{i % 3}\"\n",
    "            }\n",
    "        })\n",
    "    \n",
    "    # Upsert these vectors\n",
    "    index.upsert(vectors=timestamp_vectors)\n",
    "    time.sleep(2)  # Allow time for indexing\n",
    "    \n",
    "    # Now query with range filters\n",
    "    one_week_ago = current_time - (7 * 86400)\n",
    "    filter_range = {\"timestamp\": {\"$gte\": one_week_ago}}\n",
    "    results_range = index.query(\n",
    "        vector=query_vector,\n",
    "        filter=filter_range,\n",
    "        top_k=3,\n",
    "        include_metadata=True\n",
    "    )\n",
    "    print(\"\\n2. Range filtering (timestamp >= 1 week ago):\")\n",
    "    for i, match in enumerate(results_range.matches):\n",
    "        print(f\"  {i+1}. ID: {match.id}, Score: {match.score:.4f}\")\n",
    "        print(f\"     Timestamp: {time.ctime(match.metadata['timestamp'])}\")\n",
    "    \n",
    "    # 3. Combining filters with $and, $or\n",
    "    filter_combined = {\n",
    "        \"$and\": [\n",
    "            {\"timestamp\": {\"$gte\": one_week_ago}},\n",
    "            {\"$or\": [\n",
    "                {\"priority\": {\"$lt\": 2}},\n",
    "                {\"category\": \"cat-1\"}\n",
    "            ]}\n",
    "        ]\n",
    "    }\n",
    "    results_combined = index.query(\n",
    "        vector=query_vector,\n",
    "        filter=filter_combined,\n",
    "        top_k=3,\n",
    "        include_metadata=True\n",
    "    )\n",
    "    print(\"\\n3. Combined filtering (recent AND (high priority OR specific category)):\")\n",
    "    for i, match in enumerate(results_combined.matches):\n",
    "        print(f\"  {i+1}. ID: {match.id}, Score: {match.score:.4f}\")\n",
    "        print(f\"     Priority: {match.metadata['priority']}, Category: {match.metadata['category']}\")\n",
    "        print(f\"     Timestamp: {time.ctime(match.metadata['timestamp'])}\")\n",
    "    \n",
    "    # 4. Using $in operator for multiple values\n",
    "    filter_in = {\n",
    "        \"$and\": [\n",
    "            {\"category\": {\"$in\": [\"cat-0\", \"cat-2\"]}},\n",
    "            {\"priority\": {\"$in\": [0, 3, 4]}}\n",
    "        ]\n",
    "    }\n",
    "    results_in = index.query(\n",
    "        vector=query_vector,\n",
    "        filter=filter_in,\n",
    "        top_k=3,\n",
    "        include_metadata=True\n",
    "    )\n",
    "    print(\"\\n4. Using $in operator (category in [cat-0, cat-2] AND priority in [0, 3, 4]):\")\n",
    "    for i, match in enumerate(results_in.matches):\n",
    "        print(f\"  {i+1}. ID: {match.id}, Score: {match.score:.4f}\")\n",
    "        print(f\"     Priority: {match.metadata['priority']}, Category: {match.metadata['category']}\")\n",
    "\n",
    "# Run advanced metadata filtering examples on our multimodal index\n",
    "advanced_metadata_filtering(multimodal_index)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Query-Time Vector Transformations\n",
    "\n",
    "Sometimes you need to transform vectors at query time. This can include:\n",
    "- Normalization\n",
    "- Dimensionality reduction\n",
    "- Vector blending or interpolation\n",
    "\n",
    "Let's implement some of these techniques."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def normalize_vector(vector: List[float]) -> List[float]:\n",
    "    \"\"\"Normalize a vector to unit length.\"\"\"\n",
    "    array = np.array(vector)\n",
    "    norm = np.linalg.norm(array)\n",
    "    if norm == 0:\n",
    "        return vector\n",
    "    return (array / norm).tolist()\n",
    "\n",
    "def reduce_dimensions(vectors: List[List[float]], target_dim: int) -> List[List[float]]:\n",
    "    \"\"\"Reduce dimensionality using PCA.\"\"\"\n",
    "    pca = PCA(n_components=target_dim)\n",
    "    reduced = pca.fit_transform(vectors)\n",
    "    return reduced.tolist()\n",
    "\n",
    "def blend_vectors(vector1: List[float], vector2: List[float], weight: float = 0.5) -> List[float]:\n",
    "    \"\"\"Blend two vectors with a weight parameter.\"\"\"\n",
    "    v1 = np.array(vector1)\n",
    "    v2 = np.array(vector2)\n",
    "    blended = (weight * v1) + ((1 - weight) * v2)\n",
    "    return normalize_vector(blended.tolist())\n",
    "\n",
    "# Generate some test vectors\n",
    "test_vectors = create_random_vectors(5, 128)\n",
    "\n",
    "# 1. Normalize a vector\n",
    "normalized = normalize_vector(test_vectors[0])\n",
    "norm = np.linalg.norm(normalized)\n",
    "print(f\"Normalized vector has norm: {norm:.4f}\")\n",
    "\n",
    "# 2. Reduce dimensions\n",
    "reduced = reduce_dimensions(test_vectors, 64)\n",
    "print(f\"Original dimension: {len(test_vectors[0])}, Reduced dimension: {len(reduced[0])}\")\n",
    "\n",
    "# 3. Blend vectors\n",
    "blended = blend_vectors(test_vectors[0], test_vectors[1], 0.7)\n",
    "print(f\"Blended vector has {len(blended)} dimensions and norm: {np.linalg.norm(blended):.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Demonstration of Query-Time Vector Transformations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def query_with_transformations(index_name: str):\n",
    "    \"\"\"Demonstrate query-time vector transformations.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    index = pc.index(index_name)\n",
    "    \n",
    "    # Get stats about the index\n",
    "    stats = index.describe_index_stats()\n",
    "    print(f\"Index has {stats.namespaces.get('', {}).get('vector_count', 0)} vectors\")\n",
    "    \n",
    "    # Create test vectors to query with\n",
    "    # We'll get all text vectors to work with\n",
    "    results = index.query(\n",
    "        vector=create_random_vectors(1, 1024)[0],\n",
    "        filter={\"type\": \"text\"},\n",
    "        top_k=5,\n",
    "        include_values=True,\n",
    "        include_metadata=True\n",
    "    )\n",
    "    \n",
    "    # If we got some text vectors, let's use them\n",
    "    if results.matches:\n",
    "        text_vector = results.matches[0].values\n",
    "        desc = results.matches[0].metadata.get(\"description\", \"Unknown\")\n",
    "        print(f\"\\nUsing text vector for: '{desc}'\")\n",
    "        \n",
    "        # 1. Query with normalized vector\n",
    "        normalized = normalize_vector(text_vector)\n",
    "        results_norm = index.query(\n",
    "            vector=normalized,\n",
    "            top_k=3,\n",
    "            include_metadata=True\n",
    "        )\n",
    "        print(\"\\n1. Query with normalized vector:\")\n",
    "        for i, match in enumerate(results_norm.matches):\n",
    "            print(f\"  {i+1}. ID: {match.id}, Score: {match.score:.4f}\")\n",
    "            print(f\"     Description: {match.metadata.get('description', 'N/A')}\")\n",
    "        \n",
    "        # 2. Blend two vectors and query\n",
    "        if len(results.matches) >= 2:\n",
    "            second_vector = results.matches[1].values\n",
    "            second_desc = results.matches[1].metadata.get(\"description\", \"Unknown\")\n",
    "            print(f\"\\nBlending with second vector: '{second_desc}'\")\n",
    "            \n",
    "            blended = blend_vectors(text_vector, second_vector, 0.7)\n",
    "            results_blend = index.query(\n",
    "                vector=blended,\n",
    "                top_k=3,\n",
    "                include_metadata=True\n",
    "            )\n",
    "            print(\"\\n2. Query with blended vector (70% first, 30% second):\")\n",
    "            for i, match in enumerate(results_blend.matches):\n",
    "                print(f\"  {i+1}. ID: {match.id}, Score: {match.score:.4f}\")\n",
    "                print(f\"     Description: {match.metadata.get('description', 'N/A')}\")\n",
    "    else:\n",
    "        print(\"No text vectors found in the index\")\n",
    "\n",
    "# Run query transformation examples\n",
    "query_with_transformations(multimodal_index)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Performance Optimization\n",
    "\n",
    "Understanding how to optimize Pinecone for performance is crucial for production applications. Let's simulate and measure query performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def measure_query_performance(index_name: str, query_count: int = 10):\n",
    "    \"\"\"Measure query performance with different parameters.\"\"\"\n",
    "    pc = get_pinecone_client()\n",
    "    index = pc.index(index_name)\n",
    "    \n",
    "    # Create query vectors\n",
    "    query_vectors = create_random_vectors(query_count, 1024)\n",
    "    \n",
    "    # Scenarios to test\n",
    "    scenarios = [\n",
    "        {\"name\